## Creating new columns for raw counts and percentages and filling in the missing values

In [1]:
import pandas as pd

In [128]:
df = pd.read_excel('cleaned_K13_2_updated.xlsx')

In [130]:
import pandas as pd
import re

# Ensure string format
df["mutation_frequency(%)"] = df["mutation_frequency(%)"].astype(str)
df["data_extraction_comments"] = df["data_extraction_comments"].astype(str)

def extract_values(row):
    freq = row["mutation_frequency(%)"].strip()
    comment = row["data_extraction_comments"].strip().upper()

    # Case 1: Direct percentage e.g. 4.5
    if "PERCENTAGE" in comment:
        try:
            return pd.Series({
                "raw_count": None,
                "percentage_frequency": float(freq)
            })
        except ValueError:
            return pd.Series({
                "raw_count": None,
                "percentage_frequency": None
            })

    # Case 2: RAW_COUNTS comment
    if "RAW_COUNTS" in comment:
        # If format is x/y, calculate % from it
        if "/" in freq:
            try:
                num, denom = map(float, freq.split("/"))
                percentage = round((num / denom) * 100, 2) if denom else None
                return pd.Series({
                    "raw_count": freq,
                    "percentage_frequency": percentage
                })
            except:
                return pd.Series({
                    "raw_count": freq,
                    "percentage_frequency": None
                })
        else:
            return pd.Series({
                "raw_count": freq,
                "percentage_frequency": None
            })

    # Case 3: mixed format like 90/195 (46.4)
    match = re.match(r"(\d+/\d+)\s*\(([\d\.]+)\)", freq)
    if match:
        raw = match.group(1)
        percentage = float(match.group(2))
        return pd.Series({
            "raw_count": raw,
            "percentage_frequency": percentage
        })

    # Fallback: Try to calculate from x/y
    if "/" in freq:
        try:
            num, denom = map(float, freq.split("/"))
            percentage = round((num / denom) * 100, 2) if denom else None
            return pd.Series({
                "raw_count": freq,
                "percentage_frequency": percentage
            })
        except:
            return pd.Series({
                "raw_count": freq,
                "percentage_frequency": None
            })

    # If everything fails
    return pd.Series({
        "raw_count": None,
        "percentage_frequency": None
    })

# Apply it
df[["raw_count", "percentage_frequency"]] = df.apply(extract_values, axis=1)

In [131]:
df.head(20)

,pmid,year_pub,year_sample,country,region/province,town,participants_age,participants_age_unit,sample_size,first_line_ACT,...,type_of_mutation,mutation_frequency(%),where_sequenced,other_k13_mutations,data_extraction_comments,journal_access,Latitude,Longitude,raw_count,percentage_frequency
0,34551228,2021,2015-2019,UGANDA,NORTHERN UGANDA,GULU,≥0.5,YEARS,274,AL,...,NON-SYNONYMOUS,27,MISSING,-,RAW_COUNTS,OPEN,1.3730,32.290,27,NaN
1,34551228,2021,2015-2029,UGANDA,NORTHERN UGANDA,GULU,≥0.5,YEARS,274,AL,...,NON-SYNONYMOUS,5,MISSING,-,RAW_COUNTS,OPEN,1.3730,32.290,5,NaN
2,35703955,2022,missing,DRC,CENTRAL DRC,KINSHASHA,03-Jun,YEARS,118,ASAQ,...,MISSING,0,MISSING,"A578S, Q613E, S522C",RAW_COUNTS,OPEN,4.0380,21.759,0,NaN
3,36036611,2022,2016-2017,KENYA,WESTERN KENYA,SIAYA,<5,YEARS,340,AL,...,MISSING,0,MISSING,-,RAW_COUNTS,OPEN,0.0234,37.906,0,NaN
4,39136468,2024,2015-2023,UGANDA,EASTERN UGANDA,"TORORO, BUSIA, MBALE",>0.5,YEARS,1112,AL,...,NON-SYNONYMOUS,4,MISSING,A578S,RAW_COUNTS,OPEN,1.3730,32.290,4,NaN
5,39136468,2024,2015-2023,UGANDA,EASTERN UGANDA,"TORORO, BUSIA, MBALE",>0.5,YEARS,1112,AL,...,NON-SYNONYMOUS,3,MISSING,-,RAW_COUNTS,OPEN,1.3730,32.290,3,NaN
6,34641867,2021,2018,MOZAMBIQUE,"CABO DELGADO, TETE, ZAMBÉZIA, INHAMBANE PROVINCE","MONTEPUEZ, MOATIZE, MOPEIA, MASSINGA",0.5 - 5,YEARS,641,"AL, ASAQ",...,NaN,0,"CDC, ATLANTA","TGC469TGT, GGC548GGT",RAW_COUNTS,OPEN,18.6660,35.529,0,NaN
7,26483118,2015,2013-2014,ETHIOPIA,(AMHARA REGIONAL STATE)NORTHWEST,ADDIS ZEMEN,Jan-69,YEARS,31,AL,...,MISSING,0,"UNIVERSITY OF CALGARY, CANADA",-,RAW_COUNTS,OPEN,9.1450,40.489,0,NaN
8,26483118,2015,2013-2014,ETHIOPIA,(AMHARA REGIONAL STATE)NORTHWEST,AYKEL,Jan-69,YEARS,15,AL,...,NON-SYNONYMOUS,1(6.7),"UNIVERSITY OF CALGARY, CANADA",-,RAW_COUNTS,OPEN,9.1450,40.489,1(6.7),NaN
9,26483118,2015,2013-2014,ETHIOPIA,(AMHARA REGIONAL STATE)NORTHWEST,MAKSEGNET,Jan-69,YEARS,49,AL,...,MISSING,0,"UNIVERSITY OF CALGARY, CANADA",-,RAW_COUNTS,OPEN,9.1450,40.489,0,NaN


In [132]:
import pandas as pd
import numpy as np

# Make sure raw_count and percentage_frequency columns exist and are strings
df["raw_count"] = df["raw_count"].astype(str)
df["percentage_frequency"] = df.get("percentage_frequency", pd.Series(np.nan, index=df.index))

# Only update where percentage_frequency is missing
mask_missing = df["percentage_frequency"].isna()

# Extract percentage from raw_count where it's missing
extracted_pct = df.loc[mask_missing, "raw_count"].str.extract(r"\(([\d\.]+)\)")
df.loc[mask_missing, "percentage_frequency"] = pd.to_numeric(extracted_pct[0], errors="coerce")

# Strip the bracketed part from raw_count to keep only the main value
df["raw_count"] = df["raw_count"].str.replace(r"\s*\(.*?\)", "", regex=True).str.strip()

In [133]:
df.head(20)

,pmid,year_pub,year_sample,country,region/province,town,participants_age,participants_age_unit,sample_size,first_line_ACT,...,type_of_mutation,mutation_frequency(%),where_sequenced,other_k13_mutations,data_extraction_comments,journal_access,Latitude,Longitude,raw_count,percentage_frequency
0,34551228,2021,2015-2019,UGANDA,NORTHERN UGANDA,GULU,≥0.5,YEARS,274,AL,...,NON-SYNONYMOUS,27,MISSING,-,RAW_COUNTS,OPEN,1.3730,32.290,27,NaN
1,34551228,2021,2015-2029,UGANDA,NORTHERN UGANDA,GULU,≥0.5,YEARS,274,AL,...,NON-SYNONYMOUS,5,MISSING,-,RAW_COUNTS,OPEN,1.3730,32.290,5,NaN
2,35703955,2022,missing,DRC,CENTRAL DRC,KINSHASHA,03-Jun,YEARS,118,ASAQ,...,MISSING,0,MISSING,"A578S, Q613E, S522C",RAW_COUNTS,OPEN,4.0380,21.759,0,NaN
3,36036611,2022,2016-2017,KENYA,WESTERN KENYA,SIAYA,<5,YEARS,340,AL,...,MISSING,0,MISSING,-,RAW_COUNTS,OPEN,0.0234,37.906,0,NaN
4,39136468,2024,2015-2023,UGANDA,EASTERN UGANDA,"TORORO, BUSIA, MBALE",>0.5,YEARS,1112,AL,...,NON-SYNONYMOUS,4,MISSING,A578S,RAW_COUNTS,OPEN,1.3730,32.290,4,NaN
5,39136468,2024,2015-2023,UGANDA,EASTERN UGANDA,"TORORO, BUSIA, MBALE",>0.5,YEARS,1112,AL,...,NON-SYNONYMOUS,3,MISSING,-,RAW_COUNTS,OPEN,1.3730,32.290,3,NaN
6,34641867,2021,2018,MOZAMBIQUE,"CABO DELGADO, TETE, ZAMBÉZIA, INHAMBANE PROVINCE","MONTEPUEZ, MOATIZE, MOPEIA, MASSINGA",0.5 - 5,YEARS,641,"AL, ASAQ",...,NaN,0,"CDC, ATLANTA","TGC469TGT, GGC548GGT",RAW_COUNTS,OPEN,18.6660,35.529,0,NaN
7,26483118,2015,2013-2014,ETHIOPIA,(AMHARA REGIONAL STATE)NORTHWEST,ADDIS ZEMEN,Jan-69,YEARS,31,AL,...,MISSING,0,"UNIVERSITY OF CALGARY, CANADA",-,RAW_COUNTS,OPEN,9.1450,40.489,0,NaN
8,26483118,2015,2013-2014,ETHIOPIA,(AMHARA REGIONAL STATE)NORTHWEST,AYKEL,Jan-69,YEARS,15,AL,...,NON-SYNONYMOUS,1(6.7),"UNIVERSITY OF CALGARY, CANADA",-,RAW_COUNTS,OPEN,9.1450,40.489,1,6.7
9,26483118,2015,2013-2014,ETHIOPIA,(AMHARA REGIONAL STATE)NORTHWEST,MAKSEGNET,Jan-69,YEARS,49,AL,...,MISSING,0,"UNIVERSITY OF CALGARY, CANADA",-,RAW_COUNTS,OPEN,9.1450,40.489,0,NaN


In [137]:
df

,pmid,year_pub,year_sample,country,region/province,town,participants_age,participants_age_unit,sample_size,first_line_ACT,...,type_of_mutation,mutation_frequency(%),where_sequenced,other_k13_mutations,data_extraction_comments,journal_access,Latitude,Longitude,raw_count,percentage_frequency
0,34551228,2021,2015-2019,UGANDA,NORTHERN UGANDA,GULU,≥0.5,YEARS,274,AL,...,NON-SYNONYMOUS,27,MISSING,-,RAW_COUNTS,OPEN,1.3730,32.290,27,NaN
1,34551228,2021,2015-2029,UGANDA,NORTHERN UGANDA,GULU,≥0.5,YEARS,274,AL,...,NON-SYNONYMOUS,5,MISSING,-,RAW_COUNTS,OPEN,1.3730,32.290,5,NaN
2,35703955,2022,missing,DRC,CENTRAL DRC,KINSHASHA,03-Jun,YEARS,118,ASAQ,...,MISSING,0,MISSING,"A578S, Q613E, S522C",RAW_COUNTS,OPEN,4.0380,21.759,0,NaN
3,36036611,2022,2016-2017,KENYA,WESTERN KENYA,SIAYA,<5,YEARS,340,AL,...,MISSING,0,MISSING,-,RAW_COUNTS,OPEN,0.0234,37.906,0,NaN
4,39136468,2024,2015-2023,UGANDA,EASTERN UGANDA,"TORORO, BUSIA, MBALE",>0.5,YEARS,1112,AL,...,NON-SYNONYMOUS,4,MISSING,A578S,RAW_COUNTS,OPEN,1.3730,32.290,4,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
660,37611122,2023,2022,UGANDA,RUKIGA,RUKIGA,missing,-,100,AL,...,NON-SYNONYMOUS,5.1,MISSING,-,PERCENTAGE,OPEN,1.3730,32.290,NaN,5.1
661,37611122,2023,2022,UGANDA,RUKIGA,RUKIGA,missing,-,100,AL,...,NON-SYNONYMOUS,22.6,MISSING,-,PERCENTAGE,OPEN,1.3730,32.290,NaN,22.6
662,37611122,2023,2022,UGANDA,TORORO,TORORO,missing,-,100,AL,...,NON-SYNONYMOUS,9.8,MISSING,-,PERCENTAGE,OPEN,1.3730,32.290,NaN,9.8
663,37611122,2023,2022,UGANDA,TORORO,TORORO,missing,-,100,AL,...,NON-SYNONYMOUS,15.8,MISSING,-,PERCENTAGE,OPEN,1.3730,32.290,NaN,15.8


In [139]:
# Set percentage to 0 where mutation_frequency is '0'
df.loc[df["mutation_frequency(%)"].str.strip() == "0", "percentage_frequency"] = 0.0

In [140]:
df

,pmid,year_pub,year_sample,country,region/province,town,participants_age,participants_age_unit,sample_size,first_line_ACT,...,type_of_mutation,mutation_frequency(%),where_sequenced,other_k13_mutations,data_extraction_comments,journal_access,Latitude,Longitude,raw_count,percentage_frequency
0,34551228,2021,2015-2019,UGANDA,NORTHERN UGANDA,GULU,≥0.5,YEARS,274,AL,...,NON-SYNONYMOUS,27,MISSING,-,RAW_COUNTS,OPEN,1.3730,32.290,27,NaN
1,34551228,2021,2015-2029,UGANDA,NORTHERN UGANDA,GULU,≥0.5,YEARS,274,AL,...,NON-SYNONYMOUS,5,MISSING,-,RAW_COUNTS,OPEN,1.3730,32.290,5,NaN
2,35703955,2022,missing,DRC,CENTRAL DRC,KINSHASHA,03-Jun,YEARS,118,ASAQ,...,MISSING,0,MISSING,"A578S, Q613E, S522C",RAW_COUNTS,OPEN,4.0380,21.759,0,0.0
3,36036611,2022,2016-2017,KENYA,WESTERN KENYA,SIAYA,<5,YEARS,340,AL,...,MISSING,0,MISSING,-,RAW_COUNTS,OPEN,0.0234,37.906,0,0.0
4,39136468,2024,2015-2023,UGANDA,EASTERN UGANDA,"TORORO, BUSIA, MBALE",>0.5,YEARS,1112,AL,...,NON-SYNONYMOUS,4,MISSING,A578S,RAW_COUNTS,OPEN,1.3730,32.290,4,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
660,37611122,2023,2022,UGANDA,RUKIGA,RUKIGA,missing,-,100,AL,...,NON-SYNONYMOUS,5.1,MISSING,-,PERCENTAGE,OPEN,1.3730,32.290,NaN,5.1
661,37611122,2023,2022,UGANDA,RUKIGA,RUKIGA,missing,-,100,AL,...,NON-SYNONYMOUS,22.6,MISSING,-,PERCENTAGE,OPEN,1.3730,32.290,NaN,22.6
662,37611122,2023,2022,UGANDA,TORORO,TORORO,missing,-,100,AL,...,NON-SYNONYMOUS,9.8,MISSING,-,PERCENTAGE,OPEN,1.3730,32.290,NaN,9.8
663,37611122,2023,2022,UGANDA,TORORO,TORORO,missing,-,100,AL,...,NON-SYNONYMOUS,15.8,MISSING,-,PERCENTAGE,OPEN,1.3730,32.290,NaN,15.8


In [143]:
# Set raw_count to 0 where percentage_frequency is 0
df.loc[df["percentage_frequency"] == 0, "raw_count"] = 0

In [144]:
df.to_excel("cleaned_k13_2_stripped.xlsx", index=False)

In [154]:
df=pd.read_excel("cleaned_k13_2_stripped.xlsx")

In [155]:
df

,pmid,year_pub,year_sample,country,region/province,town,participants_age,participants_age_unit,sample_size,first_line_ACT,...,type_of_mutation,mutation_frequency(%),where_sequenced,other_k13_mutations,data_extraction_comments,journal_access,Latitude,Longitude,raw_count,percentage_frequency
0,34551228,2021,2015-2019,UGANDA,NORTHERN UGANDA,GULU,≥0.5,YEARS,274,AL,...,NON-SYNONYMOUS,27,MISSING,-,RAW_COUNTS,OPEN,1.3730,32.290,27,NaN
1,34551228,2021,2015-2029,UGANDA,NORTHERN UGANDA,GULU,≥0.5,YEARS,274,AL,...,NON-SYNONYMOUS,5,MISSING,-,RAW_COUNTS,OPEN,1.3730,32.290,5,NaN
2,35703955,2022,missing,DRC,CENTRAL DRC,KINSHASHA,03-Jun,YEARS,118,ASAQ,...,MISSING,0,MISSING,"A578S, Q613E, S522C",RAW_COUNTS,OPEN,4.0380,21.759,0,0.0
3,36036611,2022,2016-2017,KENYA,WESTERN KENYA,SIAYA,<5,YEARS,340,AL,...,MISSING,0,MISSING,-,RAW_COUNTS,OPEN,0.0234,37.906,0,0.0
4,39136468,2024,2015-2023,UGANDA,EASTERN UGANDA,"TORORO, BUSIA, MBALE",>0.5,YEARS,1112,AL,...,NON-SYNONYMOUS,4,MISSING,A578S,RAW_COUNTS,OPEN,1.3730,32.290,4,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
660,37611122,2023,2022,UGANDA,RUKIGA,RUKIGA,missing,-,100,AL,...,NON-SYNONYMOUS,5.1,MISSING,-,PERCENTAGE,OPEN,1.3730,32.290,NaN,5.1
661,37611122,2023,2022,UGANDA,RUKIGA,RUKIGA,missing,-,100,AL,...,NON-SYNONYMOUS,22.6,MISSING,-,PERCENTAGE,OPEN,1.3730,32.290,NaN,22.6
662,37611122,2023,2022,UGANDA,TORORO,TORORO,missing,-,100,AL,...,NON-SYNONYMOUS,9.8,MISSING,-,PERCENTAGE,OPEN,1.3730,32.290,NaN,9.8
663,37611122,2023,2022,UGANDA,TORORO,TORORO,missing,-,100,AL,...,NON-SYNONYMOUS,15.8,MISSING,-,PERCENTAGE,OPEN,1.3730,32.290,NaN,15.8


In [156]:
import pandas as pd
import numpy as np

# Load your data here (example):
# df = pd.read_csv("your_file.csv")

# Step 1: Clean column names
df.columns = df.columns.str.strip().str.lower()

# Step 2: Replace empty strings with NaN in the three relevant columns
for col in ['raw_count', 'percentage_frequency', 'samples_genotyped']:
    df[col] = df[col].replace('', np.nan)

# Step 3: Convert to numeric (in case strings or mixed types exist)
df['raw_count'] = pd.to_numeric(df['raw_count'], errors='coerce')
df['percentage_frequency'] = pd.to_numeric(df['percentage_frequency'], errors='coerce')
df['samples_genotyped'] = pd.to_numeric(df['samples_genotyped'], errors='coerce')

# Step 4: Calculate missing percentage freq
mask_freq_missing = df['percentage_frequency'].isna() & df['raw_count'].notna() & df['samples_genotyped'].notna()
df.loc[mask_freq_missing, 'percentage_frequency'] = (
    df.loc[mask_freq_missing, 'raw_count'] / df.loc[mask_freq_missing, 'samples_genotyped']
) * 100

# Step 5: Calculate missing raw counts
mask_raw_missing = df['raw_count'].isna() & df['percentage_frequency'].notna() & df['samples_genotyped'].notna()
df.loc[mask_raw_missing, 'raw_count'] = round(
    (df.loc[mask_raw_missing, 'percentage_frequency'] / 100) * df.loc[mask_raw_missing, 'samples_genotyped']
)

# Optional: round percentage freq and convert raw counts to Int (nullable)
df['percentage_frequency'] = df['percentage_frequency'].round(2)
df['raw_count'] = df['raw_count'].astype('Int64')  # allows NA

# ✅ Done!
print(df.head())


        pmid  year_pub year_sample country  region/province  \
0   34551228      2021   2015-2019  UGANDA  NORTHERN UGANDA   
1   34551228      2021   2015-2029  UGANDA  NORTHERN UGANDA   
2   35703955      2022     missing     DRC      CENTRAL DRC   
3   36036611      2022   2016-2017   KENYA    WESTERN KENYA   
4   39136468      2024   2015-2023  UGANDA   EASTERN UGANDA   

                   town participants_age participants_age_unit sample_size  \
0                  GULU             ≥0.5                 YEARS         274   
1                  GULU             ≥0.5                 YEARS         274   
2             KINSHASHA           03-Jun                 YEARS         118   
3                 SIAYA               <5                 YEARS         340   
4  TORORO, BUSIA, MBALE             >0.5                 YEARS        1112   

  first_line_act  ... type_of_mutation mutation_frequency(%)  where_sequenced  \
0             AL  ...   NON-SYNONYMOUS                    27          M

In [157]:
df.to_excel("cleaned_k13_2_latest.xlsx", index=False)

In [158]:
print("Rows eligible to calculate raw_count:")
print(df[mask_raw_missing][['percentage_frequency', 'samples_genotyped']])


Rows eligible to calculate raw_count:
     percentage_frequency  samples_genotyped
160                  4.55               66.0
161                  1.52               66.0
164                  0.02               66.0
165                  0.02               66.0
167                  0.24              422.0
..                    ...                ...
659                  2.20               46.0
660                  5.10               59.0
661                 22.60               53.0
662                  9.80               51.0
663                 15.80               57.0

[148 rows x 2 columns]


In [159]:
# Step 5: Calculate missing raw counts
mask_raw_missing = df['raw_count'].isna() & df['percentage_frequency'].notna() & df['samples_genotyped'].notna()

print(f"Rows with missing raw_count and available percentage_frequency + samples_genotyped: {mask_raw_missing.sum()}")

if mask_raw_missing.sum() > 0:
    df.loc[mask_raw_missing, 'raw_count'] = round(
        (df.loc[mask_raw_missing, 'percentage_frequency'] / 100) * df.loc[mask_raw_missing, 'samples_genotyped']
    )
else:
    print("No rows found where raw_count can be calculated.")

Rows with missing raw_count and available percentage_frequency + samples_genotyped: 0
No rows found where raw_count can be calculated.
